In [ ]:
import numpy as np
import math
import pylhef
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
lhfile1 = pylhef.read('madgraph_ttbar_tWb.lhe')
lhfile2 = pylhef.read('madgraph_ttbar_PS_tWb.lhe')

In [ ]:
Tmom_t1 = []    #transverse momentum of top quark
Tmom_t2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 6:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_t1.append(momentum)
            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 6:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_t2.append(momentum)

bins = np.linspace(0,1200,70)         
plt.hist(Tmom_t1, bins, alpha=0.5, label='top momentum')
plt.hist(Tmom_t2, bins, alpha=0.5, label='top momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of Top Quark')


In [ ]:
Tmom_W1 = []    #transverse momentum of top quark
Tmom_W2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 24:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_W1.append(momentum)
            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 24:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_W2.append(momentum)

bins = np.linspace(0,1000,70)         
plt.hist(Tmom_W1, bins, alpha=0.5, label='W momentum')
plt.hist(Tmom_W2, bins, alpha=0.5, label='W momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of W boson')

In [ ]:
Tmom_b1 = []    #transverse momentum of top quark
Tmom_b2 = []    #transverse momentum of top quark

for event in lhfile1.events:      
    particles=event.particles      #finds the index of the top quark
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_b1.append(momentum)
            
for event in lhfile2.events:
    particles=event.particles
    for particle in particles:
        if particle.id == 5:
            x = particle.p[1]
            y = particle.p[2]
            momentum = math.sqrt(x**2+y**2)
            Tmom_b2.append(momentum)

bins = np.linspace(0,700,70)         
plt.hist(Tmom_b1, bins, alpha=0.5, label='b momentum')
plt.hist(Tmom_b2, bins, alpha=0.5, label='b momentum PS')
plt.legend(loc='upper right')
plt.xlabel('Transverse Momentum'), plt.ylabel('frequency')
plt.title('Transverse Momentum of bottom quark')